# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (we will use langchain's ChatOpenAI wrapper for convienience here.)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [4]:
from llama_index.agent.utils import FunctionMessage, monkey_patch_langchain

# TODO: right now langchain does not support function messages
#       monkey patch it to support it
monkey_patch_langchain()

In [5]:
import json
from typing import Sequence

from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory
from llama_index.tools import BaseTool, FunctionTool

Let's define some very simple calculator tools for our agent.

In [6]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [7]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

## Agent Definition

Now, we define our agent that's capable of holding a conversation and calling tools in **under 50 lines of code**.

The meat of the agent logic is in the `chat` method. At a high-level, there are 3 steps:
1. Call OpenAI to decide which tool (if any) to call and with what arguments.
2. Call the tool with the arguments to obtain an output
3. Call OpenAI to synthesize a response from the conversation context and the tool output.

The `reset` method simply resets the conversation context, so we can start another conversation.

In [45]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: ChatOpenAI = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613"),
        chat_history: ChatMessageHistory = ChatMessageHistory(),
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history.clear()

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.add_user_message(message)
        functions = [tool.metadata.to_openai_function() for _, tool in self._tools.items()]

        ai_message = self._llm.predict_messages(chat_history.messages, functions=functions)
        chat_history.add_message(ai_message)

        function_call = ai_message.additional_kwargs.get("function_call", None)
        if function_call is not None:
            function_message = self._call_function(function_call)
            chat_history.add_message(function_message)
            ai_message = self._llm.predict_messages(chat_history.messages)
            chat_history.add_message(ai_message)

        return ai_message.content

    def _call_function(self, function_call: dict) -> FunctionMessage:
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return FunctionMessage(
            name=function_call["name"],
            content=str(output), 
        )

## Let's Try It Out!

In [46]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [47]:
agent.chat('Hi')

'Hello! How can I assist you today?'

In [51]:
agent.chat('What is 2123 * 215123')

'The product of 2123 multiplied by 215123 is 456,706,129.'

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports async endpoints
* it supports callback and tracing

In [3]:
from llama_index.agent import OpenAIAgent

In [8]:
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], verbose=True)

In [9]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  What's 212 * 122 + 213. Make sure to use tools for any calculation


=== Calling Function ===
Calling function: multiply with args: {
  "a": 212,
  "b": 122
}
Got output: 25864
=== Calling Function ===
Calling function: add with args: {
  "a": 25864,
  "b": 213
}
Got output: 26077
Assistant: The result of 212 * 122 + 213 is 26077.



Human:  exit
